In [1]:
import logging
import asyncio
import pandas as pd
import polars as pl
from IPython.display import display, Markdown

from datetime import datetime, timedelta

import influxdb_client

from tastytrade.common.logging import setup_logging
from tastytrade.config.enumerations import Channels
from tastytrade.connections.sockets import DXLinkManager
from tastytrade.connections import Credentials, InfluxCredentials

from tastytrade.messaging.processors import TelegrafHTTPEventProcessor, RedisEventProcessor
from tastytrade.config import RedisConfigManager
from tastytrade.connections.subscription import RedisSubscriptionStore
from tastytrade.utils.time_series import forward_fill

# Show all rows in pandas DataFrames
pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)
pd.set_option("display.max_colwidth", None)

logging.getLogger().handlers.clear()

TEST = True
ENV = "Live"
DURATION = 15

EDT = 5

start_time = datetime(2025, 1, 1)

setup_logging(
    level=logging.INFO,
    log_dir="../logs",
    filename_prefix=f"{'dev' if TEST else 'prod'}_tastytrade",
    console=True,
    file=True,
)

loop = asyncio.get_event_loop()
loop.set_debug(True)
logging.getLogger("asyncio").setLevel(logging.DEBUG)

2025-03-13 14:26:20 - INFO:root:62:Logging initialized - writing to ../logs/dev_tastytrade_20250313.log


# Service Connections

In [2]:
config = RedisConfigManager(env_file="/workspace/.env")
config.initialize(force=True)

credentials = Credentials(config=config, env="Live")

dxlink = DXLinkManager(subscription_store=RedisSubscriptionStore())
await dxlink.open(credentials=credentials)

influx_user = InfluxCredentials(config=config)
influxdb = influxdb_client.InfluxDBClient(
    url=influx_user.url, token=influx_user.token, org=influx_user.org
)

for handler in dxlink.router.handler.values():
    handler.add_processor(TelegrafHTTPEventProcessor())
    handler.add_processor(RedisEventProcessor())

2025-03-13 14:26:22 - INFO:tastytrade.config.manager:170:Initialized 16 variables from .env file in Redis
2025-03-13 14:26:22 - DEBUG:asyncio:848:Get address info api.tastyworks.com:443, type=<SocketKind.SOCK_STREAM: 1>, flags=<AddressInfo.AI_ADDRCONFIG: 32>
2025-03-13 14:26:22 - DEBUG:asyncio:858:Getting address info api.tastyworks.com:443, type=<SocketKind.SOCK_STREAM: 1>, flags=<AddressInfo.AI_ADDRCONFIG: 32> took 17.325ms: [(<AddressFamily.AF_INET: 2>, <SocketKind.SOCK_STREAM: 1>, 6, '', ('170.76.244.144', 443))]
2025-03-13 14:26:22 - DEBUG:asyncio:535:<asyncio.sslproto.SSLProtocol object at 0x7bcace62d410> starts SSL handshake
2025-03-13 14:26:22 - DEBUG:asyncio:594:<asyncio.sslproto.SSLProtocol object at 0x7bcace62d410>: SSL handshake took 41.9 ms
2025-03-13 14:26:22 - DEBUG:asyncio:1121:<asyncio.TransportSocket fd=85, family=2, type=1, proto=6, laddr=('172.18.0.7', 42526), raddr=('170.76.244.144', 443)> connected to None:None: (<asyncio.sslproto._SSLProtocolTransport object at 0

2025-03-13 14:26:23 - INFO:tastytrade.messaging.handlers:212:SETUP
2025-03-13 14:26:23 - INFO:tastytrade.messaging.handlers:215:AUTH_STATE:UNAUTHORIZED
2025-03-13 14:26:23 - INFO:tastytrade.messaging.handlers:215:AUTH_STATE:AUTHORIZED
2025-03-13 14:26:23 - INFO:tastytrade.messaging.handlers:218:CHANNEL_OPENED:1
2025-03-13 14:26:23 - INFO:tastytrade.messaging.handlers:218:CHANNEL_OPENED:3
2025-03-13 14:26:23 - INFO:tastytrade.messaging.handlers:218:CHANNEL_OPENED:5
2025-03-13 14:26:23 - INFO:tastytrade.messaging.handlers:218:CHANNEL_OPENED:7
2025-03-13 14:26:23 - INFO:tastytrade.messaging.handlers:218:CHANNEL_OPENED:9
2025-03-13 14:26:23 - INFO:tastytrade.messaging.handlers:218:CHANNEL_OPENED:11
2025-03-13 14:26:23 - INFO:tastytrade.messaging.handlers:218:CHANNEL_OPENED:99
2025-03-13 14:26:23 - INFO:tastytrade.messaging.handlers:223:FEED_CONFIG:5:COMPACT
2025-03-13 14:26:23 - INFO:tastytrade.messaging.handlers:223:FEED_CONFIG:7:COMPACT
2025-03-13 14:26:23 - INFO:tastytrade.messaging.han

# Market Data Subscriptions

In [3]:
start_time = datetime(2025, 3, 13)

symbols = ["BTC/USD:CXTALP", "NVDA", "AAPL", "QQQ", "SPY", "SPX", "/ESH25:XCME"]
intervals = ["1d", "1h", "30m", "15m", "5m", "m"]

# ticker subscriptions
await dxlink.subscribe(symbols)

# candle subscriptions
for symbol in symbols:
    for interval in intervals:
        coroutine = dxlink.subscribe_to_candles(
            symbol=symbol,
            interval=interval,
            from_time=start_time,
        )
        await asyncio.wait_for(coroutine, timeout=10)

2025-03-13 14:26:24 - INFO:tastytrade.connections.sockets:231:Added subscription: BTC/USD:CXTALP
2025-03-13 14:26:24 - INFO:tastytrade.connections.sockets:231:Added subscription: NVDA
2025-03-13 14:26:24 - INFO:tastytrade.connections.sockets:231:Added subscription: AAPL
2025-03-13 14:26:24 - INFO:tastytrade.connections.sockets:231:Added subscription: QQQ
2025-03-13 14:26:24 - INFO:tastytrade.connections.sockets:231:Added subscription: SPY
2025-03-13 14:26:24 - INFO:tastytrade.connections.sockets:231:Added subscription: SPX
2025-03-13 14:26:24 - INFO:tastytrade.connections.sockets:231:Added subscription: /ESH25:XCME
2025-03-13 14:26:24 - INFO:tastytrade.connections.sockets:231:Added subscription: BTC/USD:CXTALP{=d}
2025-03-13 14:26:24 - INFO:tastytrade.connections.sockets:231:Added subscription: BTC/USD:CXTALP{=h}
2025-03-13 14:26:24 - INFO:tastytrade.connections.sockets:231:Added subscription: BTC/USD:CXTALP{=30m}
2025-03-13 14:26:24 - INFO:tastytrade.connections.sockets:231:Added subs

In [4]:
# forward fill
for symbol in symbols:
    for interval in intervals:
        event_symbol = f"{symbol}{{={interval}}}"
        logging.debug("Forward-filling %s", event_symbol)
        forward_fill(symbol=event_symbol, lookback_days=5)

# Check | Market Data feeds

In [5]:
symbol = "SPX{=m}"
symbol = "BTC/USD:CXTALP{=5m}"
symbol = "/ESH25:XCME{=m}"

display(Markdown(f"**Candle Feed:** {symbol}"))
display(
    dxlink.router.handler[Channels.Candle]
    .processors["feed"]
    .frames[f"{symbol}"]
    .tail(5)
    .sort(by="time", descending=True)
)

**Candle Feed:** /ESH25:XCME{=m}

eventSymbol,time,eventFlags,index,sequence,count,open,high,low,close,volume,bidVolume,askVolume,openInterest,vwap,impVolatility
str,datetime[μs],i64,i64,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""/ESH25:XCME{=m}""",2025-03-13 14:26:00,0,7481300281888604160,0,1650,5598.75,5600.75,5597.25,5598.25,2093.0,1095.0,998.0,2.091047e6,5599.07,0.26
"""/ESH25:XCME{=m}""",2025-03-13 14:25:00,0,7481300024190566400,0,6041,5596.75,5603.75,5595.0,5598.5,8133.0,3837.0,4296.0,2.091047e6,5600.28,0.26
"""/ESH25:XCME{=m}""",2025-03-13 14:24:00,0,7481299766492528640,0,3969,5590.25,5597.5,5588.0,5596.75,4878.0,2022.0,2856.0,2.091047e6,5591.91,0.26
"""/ESH25:XCME{=m}""",2025-03-13 14:23:00,0,7481299508794490880,0,4162,5591.0,5591.5,5587.5,5590.0,5128.0,2720.0,2408.0,2.091047e6,5589.4,0.26
"""/ESH25:XCME{=m}""",2025-03-13 14:22:00,0,7481299251096453120,0,2732,5593.25,5593.5,5589.5,5590.75,3361.0,1666.0,1695.0,2.091047e6,5591.22,0.26


In [6]:
display(Markdown(f"**Quote Feed:** {symbol}"))

display(dxlink.router.handler[Channels.Quote].processors["feed"].pl)

**Quote Feed:** /ESH25:XCME{=m}

eventSymbol,bidPrice,askPrice,bidSize,askSize
str,f64,f64,f64,f64
"""BTC/USD:CXTALP""",81944.53,82520.18,1.5,1.5
"""SPX""",5590.98,5596.66,null,null
"""SPY""",558.54,558.56,201.0,300.0
"""QQQ""",475.56,475.57,120.0,18.0
"""NVDA""",117.47,117.49,420.0,350.0
"""/ESH25:XCME""",5598.75,5599.0,6.0,31.0
"""AAPL""",215.2,215.23,143.0,339.0


In [7]:
# TODOS

# [x] Widen the plot
# [x] Remove the scroller at the bottom
# [x] move the legent and remove Price (that is obvious)
# [x] Add MACD
# [x] Fix HULL - Align w/ candlesticks
# [x] ERROR if no study data found

# [ ] Add RSI
# [ ] Add Volume Profile (?? ... /ES, SPY, etc)
# [ ] Add velocity metric

In [ ]:
from tastytrade.analytics.visualizations.plots import (
    plot_macd_with_hull,
    HorizontalLine,
    VerticalLine,
)
from tastytrade.analytics.indicators.momentum import macd
from tastytrade.messaging.models.events import CandleEvent
import re

from tastytrade.providers.market import MarketDataProvider
from tastytrade.providers.subscriptions import RedisSubscription
from tastytrade.config import RedisConfigManager
from tastytrade.analytics.visualizations.utils import get_opening_range

import pytz

month = 3
day = 13

subscription = RedisSubscription(config=RedisConfigManager())
await subscription.connect()

et_tz = pytz.timezone("America/New_York")
market_open = datetime(2025, month, day, 9, 30, tzinfo=et_tz)
morning_end = datetime(2025, month, day, 11, 30, tzinfo=et_tz)
lunch_end = datetime(2025, month, day, 13, 30, tzinfo=et_tz)
market_close = datetime(2025, month, day, 16, 0, tzinfo=et_tz)

streamer = MarketDataProvider(subscription, influxdb)

start = market_open + timedelta(minutes=-(padding := 5))
stop = market_close + timedelta(minutes=padding)

In [42]:
candle_symbol = "SPX{=m}"
# candle_symbol = "/ESH25:XCME{=m}"

prior_day: CandleEvent = CandleEvent(
    **(
        streamer.download(
            symbol=re.sub(r"\{=.*?\}", "{=d}", candle_symbol),
            start=start.date() + timedelta(days=-1),
            stop=start.date(),
            debug_mode=True,
        )
        .to_dicts()
        .pop()
    )
)

In [ ]:
or5 = await get_opening_range(
    streamer,
    "SPX{=m}",
    5,
    date=start.date(),
)

or15 = await get_opening_range(
    streamer,
    "SPX{=m}",
    15,
    date=start.date(),
)

or30 = await get_opening_range(
    streamer,
    "SPX{=m}",
    30,
    date=start.date(),
)

In [44]:
levels = [
    HorizontalLine(
        price=prior_day.close,
        color="#FF66FE",  # Orange
        line_dash="dot",
        label_font_size=10.5,
        opacity=0.45,
    ),
    HorizontalLine(
        price=prior_day.high,
        color="#4CAF50",  # Green
        line_dash="dot",
        label_font_size=10.5,
        opacity=0.45,
    ),
    HorizontalLine(
        price=prior_day.low,
        color="#F44336",  # Red
        line_dash="dot",
        label_font_size=10.5,
        opacity=0.45,
    ),
    HorizontalLine(
        price=or5.high,
        start_time=market_open,
        color="#4CAF50",  # Green
        line_dash="solid",
        opacity=0.75,
    ),
    HorizontalLine(
        price=or5.low,
        start_time=market_open,
        color="#4CAF50",  # Green
        line_dash="solid",
        opacity=0.75,
    ),
    HorizontalLine(
        price=or15.high,
        start_time=market_open + timedelta(minutes=15),
        color="#4CAF50",  # Green
        line_dash="solid",
        opacity=0.45 if or15.high != or5.high else 0.0,
    ),
    HorizontalLine(
        price=or15.low,
        start_time=market_open + timedelta(minutes=15),
        color="#4CAF50",  # Green
        line_dash="solid",
        opacity=0.45 if or15.low != or5.low else 0.0,
    ),
    HorizontalLine(
        price=or30.high,
        start_time=market_open + timedelta(minutes=30),
        color="#4CAF50",  # Green
        line_dash="dot",
        opacity=0.45 if or30.high != or15.high else 0.0,
    ),
    HorizontalLine(
        price=or30.low,
        start_time=market_open + timedelta(minutes=30),
        color="#4CAF50",  # Green
        line_dash="dot",
        opacity=0.45 if or30.low != or15.low else 0.0,
    ),
]

In [45]:
executions = [
    VerticalLine(
        time=datetime(2025, 2, 26, 14, 30) + timedelta(hours=1, minutes=50),
        color="#555555",
        line_dash="dot",
        label="Open",
    ),
    VerticalLine(
        time=datetime(2025, 2, 26, 14, 30) + timedelta(hours=4, minutes=50),
        color="#555555",
        line_dash="dot",
        label="Close",
    ),
    VerticalLine(
        time=datetime(2025, 2, 27, 14, 30) + timedelta(minutes=20),
        color="#555555",
        line_dash="dot",
        label="Open",
    ),
    VerticalLine(
        time=datetime(2025, 2, 28, 14, 30) + timedelta(hours=1, minutes=40),
        color="#555555",
        line_dash="dot",
        label="Open",
    ),
    VerticalLine(
        time=datetime(2025, 2, 28, 14, 30) + timedelta(hours=4, minutes=0),
        color="#555555",
        line_dash="dot",
        label="Close",
    ),
    VerticalLine(
        time=datetime(2025, 3, 3, 14, 30) + timedelta(hours=0, minutes=20),
        color="#555555",
        line_dash="dot",
        label="Open",
    ),
    VerticalLine(
        time=datetime(2025, 3, 4, 14, 30) + timedelta(hours=1, minutes=10),
        color="#555555",
        line_dash="dot",
        label="Pass",
    ),
    VerticalLine(
        time=datetime(2025, 3, 4, 14, 30) + timedelta(hours=2, minutes=10),
        color="#555555",
        line_dash="dot",
        label="Open",
    ),
    VerticalLine(
        time=datetime(2025, 3, 6, 14, 30) + timedelta(hours=0, minutes=45),
        color="#555555",
        line_dash="dot",
        label="Pass",
    ),
    VerticalLine(
        time=datetime(2025, 3, 6, 14, 30) + timedelta(hours=2, minutes=10),
        color="#555555",
        line_dash="dot",
        label="Open",
    ),
    VerticalLine(
        time=datetime(2025, 3, 7, 14, 30) + timedelta(hours=0, minutes=45),
        color="#555555",
        line_dash="dot",
        label="Open",
    ),
    VerticalLine(
        time=datetime(2025, 3, 7, 14, 30) + timedelta(hours=2, minutes=55),
        color="#555555",
        line_dash="dot",
        label="Open",
    ),
    VerticalLine(
        time=datetime(2025, 3, 13, 10, 20, tzinfo=et_tz),
        color="#555555",
        line_dash="dot",
        label="Open",
    ),
]

In [ ]:
candles: pl.DataFrame = streamer.download(
    symbol=candle_symbol,
    start=start,
    stop=stop,
    debug_mode=True,
)

df_macd = macd(candles, prior_close=prior_day.close, fast_length=12, slow_length=26, macd_length=9)

plot_macd_with_hull(
    df_macd,
    pad_value=prior_day.close,
    start_time=start,
    end_time=stop,
    horizontal_lines=levels,
    vertical_lines=executions,
)

In [ ]:
candles_5m: pl.DataFrame = streamer.download(
    symbol=candle_symbol.replace("m", "5m"),
    start=start,
    stop=stop,
    debug_mode=True,
)

df_macd_5m = macd(
    candles_5m, prior_close=prior_day.close, fast_length=12, slow_length=26, macd_length=9
)

plot_macd_with_hull(
    df_macd_5m,
    pad_value=prior_day.close,
    start_time=start,
    end_time=stop,
    horizontal_lines=levels,
    vertical_lines=executions,
)

In [ ]:
# debug
df_macd_5m.to_pandas().head()

In [ ]:
await dxlink.close()